In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import numpy as np

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_feature_film_series_with_two_entries"

In [4]:
r = requests.get(url)

In [5]:
soup = BeautifulSoup(r.text, 'html.parser')

In [6]:
#mix the soup
m = [item.get_text().split('\n') for item in soup.find_all('ol')]

In [8]:
#Quick function to query the OMDB API
def make_text(text, column):
    try:
        return text[column]
    except:
        return "N/A"

counter = 0

def query_api(title, year):
    
    url = 'http://www.omdbapi.com'
    
    #Handle String years if necessary
    try:
        year = str(int(year))
    except:
        year = 'N/A'
    
    #Populate Params
    params = {'t': title,
           'y': year,
           'apikey': 'f13c3a86'}

    global counter
    counter = counter + 1
    print(counter, title)
    
    #Make the request
    try:
        r = requests.get(url, params=params, timeout=0.6)

        response = r.text
        #print(response)

        json_text = json.loads(response)
    except Exception as e:
        print(e)
        r = 0
    
    try:
        #Populate imdbRating
        '''
        try:
            imdb = json_text['imdbRating']
        except:
            imdb = 'N/A'
        '''
        imdb = make_text(json_text, 'imdbRating')

        #Populate Metascore Rating
        '''
        try:
            metascore = json_text['Metascore']
        except:
            metascore = 'N/A'
        ''' 
        metascore = make_text(json_text, 'Metascore')

        imdb_id = make_text(json_text, 'imdbID')
        #print(imdb, metascore)
    except:
        imdb = 'N/A'
        metascore = 'N/A'
        imdb_id = 'N/A'
    
    return {'imdb': imdb, 'metascore': metascore, 'imdb_id': imdb_id}

In [9]:
#Find all movies
l = [item.text.split('\n') for item in soup.find('div', {'class': 'div-col'}).find_all('li')]

In [10]:
#Remove anything that doesn't have 3 entries
'''
def filterList(item):
    return len(item) == 3
'''

clean_list = list(filter(lambda x: len(x) == 3, l))

In [11]:
#TODO: refactor to work with more than 2 movies
titles = [[{'title':item[1], 'series': item[0]}, {'title':item[2], 'series':item[0]}] for item in clean_list]

In [12]:
#Flatten the list to create a dataframe
df = pd.DataFrame([item for sublist in titles for item in sublist])

In [13]:
#Create a new column by parsing out the date
def return_date(item):
    try:
        return item.split('(')[1].split(')')[0]
    except:
        return np.nan
    
df['date'] = df['title'].apply(lambda x:return_date(x))
df['title_clean'] = df['title'].apply(lambda x: x.split('(')[0])

In [14]:
df

,title,series,date,title_clean
0,00 Schneider – Jagd auf Nihil Baxter (1994),00 Schneider,1994,00 Schneider – Jagd auf Nihil Baxter
1,00 Schneider – Im Wendekreis der Eidechse [de]...,00 Schneider,2013,00 Schneider – Im Wendekreis der Eidechse [de]
2,2 Days in Paris (2007),2 Days in...,2007,2 Days in Paris
3,2 Days in New York (2012),2 Days in...,2012,2 Days in New York
4,8mm (1999),8mm,1999,8mm
...,...,...,...,...
3347,Zulu Dawn (1979) (prequel),Zulu,1979,Zulu Dawn
3348,Zuma (1985),Zuma,1985,Zuma
3349,Anak ni Zuma (1987),Zuma,1987,Anak ni Zuma
3350,Dschungelmädchen für zwei Halunken [de] (1974),Zwei Teufelskerle,1974,Dschungelmädchen für zwei Halunken [de]


In [15]:
#Converte the date column to be and integer
def convert_to_int(item):
    try:
        return float(item)
    except:
        return np.nan

df['date'] = df['date'].apply(lambda x: convert_to_int(x))

In [16]:
#Only keep movies newer than 1990 and let's only worry about the first 100
test_df = df.loc[df['date'] > 1990].copy()

In [80]:
#test_df

In [17]:
#Query the API for each entry and keep building the table
final_df = test_df.merge(test_df.apply(lambda x: pd.Series(query_api(x['title_clean'], x['date'])), axis=1), 
             left_index=True, right_index=True)

1 00 Schneider – Jagd auf Nihil Baxter 
2 00 Schneider – Jagd auf Nihil Baxter 
3 00 Schneider – Im Wendekreis der Eidechse [de] 
4 2 Days in Paris 
5 2 Days in New York 
6 8mm 
7 8mm 2 
8 28 Days Later 
9 28 Weeks Later 
10 30 Days of Night 
11 30 Days of Night: Dark Days 
12 47 Meters Down 
13 47 Meters Down: Uncaged 
14 100 Girls 
15 100 Women 
16 101 Dalmatians II: Patch's London Adventure 
17 101 Dalmatians 
18 102 Dalmatians 
19 300 
20 300: Rise of an Empire 
21 500 Years Later 
22 Motherland 
23 666: The Child 
24 666: The Beast 
25 976-Evil II 
26 2001 Maniacs 
27 2001 Maniacs: Field of Screams 
28 A Dog's Purpose 
29 A Dog's Journey 
30 A Gift to Remember 
31 Cherished Memories: A Gift to Remember 2 
32 A Haunted House 
33 A Haunted House 2 
34 A Monkey's Tale 
35 The Prince's Voyage [fr] 
36 A Quiet Place 
37 A Quiet Place 2 
38 Aadu 
39 Aadu 2 
40 Aashiqui 2 
41 Ab Tak Chhappan 
42 Ab Tak Chhappan 2 
43 Abang '92 
44 Abang Long Fadil 
45 Abang Long Fadil 2 
46 ABCD: Any Bod

UnboundLocalError: local variable 'imdb_id' referenced before assignment

In [82]:
# Find the unique names of the series
# series_names = final_df['series'].unique()

In [ ]:
#Convert the imdb and metascore columns to int - Could probably clean this up as well

final_df['imdb'] = final_df['imdb'].apply(lambda x: convert_to_int(x))
final_df['metascore'] = final_df['metascore'].apply(lambda x: convert_to_int(x))

In [ ]:
final_df.head()

In [ ]:
# Drop NA in the imdb column and group_by series, then filter out those series that only have one entry
keep_series = pd.DataFrame(final_df.dropna(subset=['imdb']).groupby('series')['title'].count() == 1).reset_index()

# Build a list of the series to keep (This could probably be combined with the previous cell)
keep_list = list(keep_series.loc[keep_series['title'] == False, 'series'])

# Create the final data frame by keeping only the series that are in the keep list
output = final_df.loc[final_df['series'].isin(keep_list)]

In [ ]:
# Create a summary dataframe
summary = output.groupby('series')['imdb'].last() - final_df.groupby('series')['imdb'].first()

In [ ]:
# Plot the review distribution - Maybe I'll do something with this later

summary.plot.kde()

In [ ]:
summary.describe()

In [ ]:
#Write output to json format so it's easier to work with in javascript
json_data = output[['title_clean', 'series', 'date', 'imdb', 'imdb_id']].to_json(orient='records')

#Write it to the res directory
with open('../res/movieData.json', 'w') as f:
    f.write(json_data)

In [ ]:
summary_data = pd.DataFrame(summary.dropna()).reset_index().to_json(orient='records')
with open('../res/summary.json', 'w') as f:
    f.write(summary_data)